In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
edf_path="/content/drive/MyDrive/temp/signal_sync_EDF ("

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.feather as feather
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D

In [ ]:
edf = pd.DataFrame()

for i in range (1,23):
  path = edf_path + str(i) + ').feather'
  edf_temp = feather.read_table(path).to_pandas()
  edf = pd.concat([edf, edf_temp.iloc[::100,:]])

In [ ]:
psg_columns=['Snore_x', 'PulseRate','SpO2']

# psg_columns=['Snore_x', 'ECG I', 'SpO2', 'PulseRate', 'Leg 1', 'Leg 2', 'Body','EOG LOC-A2', 'EOG ROC-A2', 'EEG F3-A2', 'EEG F4-A1', 'EEG A1-A2', 'EEG C3-A2', 'EEG C4-A1', 'EEG O1-A2', 'EEG O2-A1', 'EMG Chin','Flow Patient-0', 'Flow Patient-1', 'Effort THO', 'Effort ABD']

In [ ]:
X = edf[psg_columns]
y = edf['Stage']

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
X_scaled  = (X_resampled - X_resampled.min()) / (X_resampled.max() - X_resampled.min())
X_scaled = X_scaled.pow(2)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
def context_window(x, window_size=20):
    X_context = []
    for i in range(len(x)):
        start = max(0, i - window_size)
        end = min(len(x), i + window_size + 1)
        context = x.iloc[start:end]

        context_mean = context.mean().values
        context_std = context.std().values

        features = np.concatenate([x.iloc[i].values, context_mean, context_std])
        X_context.append(features)

    return np.array(X_context)

X_context = context_window(X_scaled)

In [ ]:
X_train_ctx, X_test_ctx, y_train_ctx, y_test_ctx = train_test_split(X_context, y_resampled, test_size=0.2, random_state=0)

In [ ]:
rfCtxClf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rfCtxClf.fit(X_train_ctx, y_train_ctx)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
y_pred = rfCtxClf.predict(X_test_ctx)
print("Točnost: ", accuracy_score(y_test_ctx, y_pred))
print(confusion_matrix(y_test_ctx, y_pred))
print(classification_report(y_test_ctx, y_pred))

Točnost:  0.9669022717195691
[[15127   241     6     0    23]
 [  275 14722   470     0    19]
 [    5   433 14388     0   457]
 [    0     1     4 15226     3]
 [   32    24   545     0 14681]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     15397
           1       0.95      0.95      0.95     15486
           2       0.93      0.94      0.94     15283
           3       1.00      1.00      1.00     15234
           4       0.97      0.96      0.96     15282

    accuracy                           0.97     76682
   macro avg       0.97      0.97      0.97     76682
weighted avg       0.97      0.97      0.97     76682



In [ ]:
filename = 'rfCtx_sleep_phase_model-0709-Less.pkl'
with open(filename, 'wb') as file:
    pickle.dump(rfCtxClf, file)

In [ ]:
def reshape_data_conv(x_in, y_in, look_back=20):
    X, y = [], []
    for i in range(look_back, len(x_in) - look_back):
        X.append(x_in.iloc[i - look_back:i + 1].values)
        y.append(y_in.iloc[i])
    return np.array(X), np.array(y)

look_back = 20
X_conv, y_conv = reshape_data_conv(X_scaled, y_resampled, look_back)
X_train_conv, X_test_conv, y_train_conv, y_test_conv = train_test_split(X_conv, y_conv, test_size=0.2, random_state=42)

In [ ]:
n_features=len(psg_columns)
input_shape = ( look_back + 1, n_features)
num_classes = 5

In [ ]:
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 64)            832       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 64)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 32)             8224      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 3, 32)            0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 96)                0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
model.fit(X_train_conv, y_train_conv, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
7668/7668 [==============================] - 352s 46ms/step - loss: 0.3848 - accuracy: 0.8419 - val_loss: 0.3194 - val_accuracy: 0.8721
Epoch 2/20
7668/7668 [==============================] - 354s 46ms/step - loss: 0.2972 - accuracy: 0.8815 - val_loss: 0.2955 - val_accuracy: 0.8836
Epoch 3/20
7668/7668 [==============================] - 365s 48ms/step - loss: 0.2702 - accuracy: 0.8939 - val_loss: 0.2614 - val_accuracy: 0.8984
Epoch 4/20
7668/7668 [==============================] - 359s 47ms/step - loss: 0.2531 - accuracy: 0.9006 - val_loss: 0.2261 - val_accuracy: 0.9138
Epoch 5/20
7668/7668 [==============================] - 357s 47ms/step - loss: 0.2382 - accuracy: 0.9068 - val_loss: 0.2661 - val_accuracy: 0.8968
Epoch 6/20
7668/7668 [==============================] - 353s 46ms/step - loss: 0.2274 - accuracy: 0.9113 - val_loss: 0.2191 - val_accuracy: 0.9159
Epoch 7/20
7668/7668 [==============================] - 357s 47ms/step - loss: 0.2176 - accuracy: 0.9155 - val_loss: 0

In [ ]:
test_loss, test_acc = model.evaluate(X_test_conv, y_test_conv)

print(f"Točnost: {test_acc}")

2397/2397 [==============================] - 36s 15ms/step - loss: 0.1820 - accuracy: 0.9304
Točnost: 0.9304196834564209


In [ ]:
filename = 'conv1d_sleep_phase_model-0709-Less.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)


In [ ]:
xgbClfCtx = XGBClassifier(
    n_estimators=100,
    n_jobs=-1,
    max_depth=12,
    learning_rate=0.5
    )
xgbClfCtx.fit(X_train_ctx, y_train_ctx)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
y_pred_xgbctx = xgbClfCtx.predict(X_test_ctx)

print("Točnost: ", accuracy_score(y_test_ctx, y_pred_xgbctx))
print(confusion_matrix(y_test_ctx, y_pred_xgbctx))
print(classification_report(y_test_ctx, y_pred_xgbctx))

Točnost:  0.9782347878250437
[[15208   170     6     0    13]
 [  185 14988   298     1    14]
 [    1   287 14697     0   298]
 [    0     1     4 15229     0]
 [   15    18   358     0 14891]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     15397
           1       0.97      0.97      0.97     15486
           2       0.96      0.96      0.96     15283
           3       1.00      1.00      1.00     15234
           4       0.98      0.97      0.98     15282

    accuracy                           0.98     76682
   macro avg       0.98      0.98      0.98     76682
weighted avg       0.98      0.98      0.98     76682



In [ ]:
filename = 'xgb_sleep_phase_model-0709_ctx.pkl'
with open(filename, 'wb') as file:
    pickle.dump(xgbClfCtx, file)